In [1]:
import pandas as pd
import numpy as np
from nltk import sent_tokenize

In [3]:
from bert_serving.client import BertClient

In [4]:
bc = BertClient()

KeyboardInterrupt: 

In [5]:
import time

## BERT Embedding

In [10]:
live_result = pd.read_csv('../results_data/final_live.csv')
online_result = pd.read_csv('../results_data/final_online.csv')

In [12]:
## Note that d20101110/d20100119/d20091116/d20070314 there is no for points or against points
loop_through = live_result.id[live_result.id[live_result.id == 'd20070314'].index[0]+1:]

In [27]:
for test_id in loop_through:
    print(test_id)
    #test_id = 'd20180206'

    try_df = pd.read_csv('../For Against Scripts/for_against_scripts_bp_free_'+test_id+'.csv')

    main_points = open('../Main Points/main_points_' + test_id).read().split('\n')
    break_point = main_points.index("Against The Motion")
    for_main_points = main_points[1:break_point]
    against_main_points = main_points[break_point+1:]

    try:
        for_main_points.remove('')
    except:
        print('Nothing to be removed from for')
    try:
        against_main_points.remove('')
    except:
        print('Nothing to be removed from against')

    #start = time.time()
    encoded_for_main_points = encode_main_point(for_main_points)
    encoded_against_main_points = encode_main_point(against_main_points)
    sim_scores = [all_dim_sim_score(script, encoded_for_main_points, encoded_against_main_points) for script in try_df.script]
    #end = time.time()

    try_df['sim_scores'] = sim_scores

    try_df.to_csv('../BERT/BERT_encoded_'+ test_id +'.csv')
    

d20070207
Nothing to be removed from for
d20061213
Nothing to be removed from for
d20061129
Nothing to be removed from for
d20061017
Nothing to be removed from for
d20060927
Nothing to be removed from for


In [28]:
bc.close()

In [6]:
'''
Split each main point to indiviudal sentences and perform dc encoding to return a nested list.
'''
def encode_main_point(main_points_list):
    splited_main_point = [sent_tokenize(main_point) for main_point in main_points_list]
    return [bc.encode(sentences) for sentences in splited_main_point]

'''
Calculate all six dimensional main points sim score for one dataframe row of script
'''
def all_dim_sim_score(script, encoded_for_main_points, encoded_against_main_points):
    doc_vecs = bc.encode(sent_tokenize(script))
    dov_vecs_norms = np.linalg.norm(doc_vecs, axis=1)
    
    ## if the corpus is for the argument, return the result as positive, if against, return as negative.
    for_sim_scores = [sim_score_cal(point, doc_vecs, dov_vecs_norms) for point in encoded_for_main_points]
    against_sim_scores = [-sim_score_cal(point, doc_vecs, dov_vecs_norms) for point in encoded_against_main_points]
    for_sim_scores.extend(against_sim_scores)
    return for_sim_scores
    
    
'''
Calculate the similarity score as the top N average with the target corpus where N is the number of sentences 
in this main point.

--main_point:  a list of encoded sentences of one main point
'''
def sim_score_cal(encoded_main_point, doc_vecs, dov_vecs_norms):
    score_list = []
    for query_vec in encoded_main_point:
        scores = np.sum(query_vec * doc_vecs, axis=1) / dov_vecs_norms
        highest = scores.max()
        score_list.append(highest)

    return np.mean(score_list)

## Create Training Dataset

In [16]:
live_results = pd.read_csv('../results_data/final_live.csv')
online_results = pd.read_csv('../results_data/final_online.csv')

In [17]:
live_results.head()

,for the against side to against,for the for side to for,from the against side to for,from the against side to undecided,from the for side to against,from the for side to undecided,from undecided to against,from undecided to for,post-debate-against,post-debate-for,post-debate-undecided,pre-debate-against,pre-debate-for,pre-debate-undecided,undecided to undecided,url,winner,id
0,17,49,2,2,5,2,6,11,28,62,10,21,56,23,6,https://www.intelligencesquaredus.org/debates/...,against,d20191112
1,40,15,8,4,9,3,10,9,59,32,9,52,27,21,2,https://www.intelligencesquaredus.org/debates/...,against,d20191029
2,19,14,3,1,15,1,30,13,64,30,6,23,30,47,4,https://www.intelligencesquaredus.org/debates/...,against,d20191022
3,29,26,5,1,6,4,16,9,51,40,9,35,36,29,4,https://www.intelligencesquaredus.org/debates/...,against,d20190917
4,49,12,1,1,14,0,20,2,83,15,2,51,26,23,1,https://www.intelligencesquaredus.org/debates/...,against,d20190802


In [22]:
import ast
max_main_points = 6
data = []

In [23]:
all_id = live_results.id
for id_ in all_id:
    try:
        print(id_)
        df = pd.read_csv('../BERT/BERT_encoded_'+ id_+ '.csv')
        df = df.dropna()
        df.sim_scores = df.sim_scores.apply(lambda x: np.array(ast.literal_eval(x)))
        
        rounds_mean = df.groupby(['round']).sim_scores.apply(lambda x: np.mean(x))
        num_main_points = sum(rounds_mean.iloc[0]>0)
        rounds_mean = rounds_mean.apply(lambda x: x.tolist())

        ## insert zero to match 5 main points
        for one_round_mean in rounds_mean:
            for i in range(num_main_points, max_main_points):
                one_round_mean.insert(i,0)
            num_neg = sum(np.array(one_round_mean)<0)
            num_to_append = max_main_points - num_neg
            for i in range(num_to_append):
                one_round_mean.append(0)
        extended_list = []
        for row in rounds_mean:
            extended_list.extend(row)
        extended_list = np.abs(extended_list)

        live_label = live_results[live_results['id'] == id_].winner.iloc[0]
        online_label = online_results[online_results['id'] == id_].winner.iloc[0]
        
        one_row = []
        one_row.append(id_)
        one_row.extend(np.append(np.append(extended_list, live_label), online_label).tolist())
        data.append(one_row)
        #print(one_row)
    except:
        continue


d20191112
d20191029
d20191022
d20190917
d20190802
d20190628
d20190418
d20190413
d20190328
d20190304
d20190211
d20190131
d20190116
d20181111
d20181101
d20181011
d20180926
d20180628
d20180626
d20180623
d20180514
d20180502
d20180421
d20180417
d20180327
d20180309
d20180206
d20171207
d20171024
d20171003
d20170928
d20170606
d20170510
d20170406
d20170322
d20170301
d20170208
d20170201
d20170111
d20161207
d20161129
d20161114
d20161026
d20161013
d20160926
d20160913
d20160907
d20160608
d20160504
d20160406
d20160309
d20160301
d20160203
d20160113
d20151203
d20151118
d20151110
d20151102
d20151027
d20151014
d20150916
d20150809
d20150602
d20150526
d20150513
d20150415
d20150331
d20150311
d20150224
d20150211
d20150115
d20141203
d20141113
d20141022
d20141007
d20140930
d20140909
d20140626
d20140507
d20140409
d20140402
d20140312
d20140305
d20140227
d20140212
d20140115
d20131204
d20131120
d20131114
d20131030
d20131018
d20131016
d20130910
d20130809
d20130619
d20130508
d20130417
d20130403
d20130313
d20130213


In [24]:
data = pd.DataFrame(data)#.loc[394]

In [26]:
data = data[data.loc[:,37].notna()]

In [27]:
data.shape

(126, 39)

In [60]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression


X = data.loc[:, 1:36]
y = data.loc[:, 37]
y = (y == 'for')

#clf = LogisticRegression(random_state=0, solver='lbfgs', max_iter=1000, multi_class='ovr')
clf = LogisticRegression(random_state=0)
scores = cross_val_score(clf, X, y, cv=5)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [65]:
print(scores.mean())
print(scores.std())
print(scores)

0.5944615384615385
0.06255771299864869
[0.69230769 0.56       0.64       0.52       0.56      ]


In [51]:
pd.Series(y_test).value_counts()

False    25
True     17
Name: 37, dtype: int64

## Acknowledgement:
https://medium.com/@adriensieg/text-similarities-da019229c894

https://github.com/hanxiao/bert-as-service